In [1]:
from cv2 import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import os.path

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers, models

from pprint import pprint as pp

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


2021-11-23 18:58:04.772601: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 18:58:04.779660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 18:58:04.780069: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Create Data df

In [3]:
TRAIN_DIR = 'DATASET_A_AP/train'
TEST_DIR = 'DATASET_A_AP/test'

# lr = 0.001
lr = 0.001
epoch = 10
batch_size = 128


In [4]:
# Convert folder to dataframe of images' paths & labels
def get_paths_labels(path, allowed_extension="jpg"):
        global Path
        images_dir = Path(path)
        
        filepaths = pd.Series((images_dir.glob(fr'**/*.{allowed_extension}'))).astype(str)
        filepaths.name = "path"
        
        labels = filepaths.str.split("/")[:].str[-2]
        labels.name = "label"

        # Concatenate filepaths and labels
        df = pd.concat([filepaths, labels], axis=1)

        # Shuffle the DataFrame and reset index
        df = df.sample(frac=1).reset_index(drop = True)
        return df

In [5]:
train_df = get_paths_labels(TRAIN_DIR)
test_df = get_paths_labels(TEST_DIR)

In [6]:
train_df

,path,label
0,DATASET_A_AP/train/X/TRAIN_X_828.jpg,X
1,DATASET_A_AP/train/X/TRAIN_X_559.jpg,X
2,DATASET_A_AP/train/S/TRAIN_S_2067.jpg,S
3,DATASET_A_AP/train/I/TRAIN_I_90.jpg,I
4,DATASET_A_AP/train/U/TRAIN_U_830.jpg,U
...,...,...
57712,DATASET_A_AP/train/A/TRAIN_A_1352.jpg,A
57713,DATASET_A_AP/train/C/TRAIN_C_1108.jpg,C
57714,DATASET_A_AP/train/Z/TRAIN_Z_1429.jpg,Z
57715,DATASET_A_AP/train/Y/TRAIN_Y_1775.jpg,Y


# Create data generator and use flow_from_dataframe

In [6]:
data_generator = ImageDataGenerator(validation_split=0.2, rescale=1./255.,
                                   zoom_range=0.1,
                                    width_shift_range=0.1,height_shift_range=0.1,
                                    shear_range=0.1,
                                   )
test_generator = ImageDataGenerator(rescale=1./255.)

train_images = data_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training'
)

val_images = data_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='validation'
)


test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    # batch_size=1,
    shuffle=False,
)

Found 46174 validated image filenames belonging to 26 classes.
Found 11543 validated image filenames belonging to 26 classes.
Found 14430 validated image filenames belonging to 26 classes.


# Compare Model

In [7]:
from inspect import getmembers, isfunction
from asl_model import tool

model_dict = dict()

print(f"model candidate = {len(tool.all_models.items())}")

for model_func_name, model_func_obj in tool.all_models.items():
    # if 'stn_a_model_8' not in model_func_name:
    #     continue
    
    print(f"\nmodel_func_name : {model_func_name}")
    model = model_func_obj()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate = lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print('\nTraining...')
    history = model.fit(
        train_images,
        validation_data=val_images,
        epochs=epoch,
    )
    
    print('\nEvaluating...')
    result = model.evaluate(test_images)
    print(f"{model_func_name} : test loss = {round(result[0], 2)}, test acc= {round(result[1], 2)}")
    
    acc = result[1]
    
    record = {
        'acc': acc,
        'history': history,
    }
    
    model_dict[model_func_name] = record

model candidate = 12

model_func_name : get_model_1


2021-11-23 00:42:07.665616: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-23 00:42:07.667011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 00:42:07.668050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 00:42:07.668966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer


Training...
Epoch 1/10


2021-11-23 00:42:09.214791: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204


361/361 [==============================] - 23s 57ms/step - loss: 0.5710 - accuracy: 0.8293 - val_loss: 2.8451 - val_accuracy: 0.2148
Epoch 2/10
361/361 [==============================] - 21s 57ms/step - loss: 0.1574 - accuracy: 0.9524 - val_loss: 0.1902 - val_accuracy: 0.9415
Epoch 3/10
361/361 [==============================] - 21s 57ms/step - loss: 0.1032 - accuracy: 0.9676 - val_loss: 0.1089 - val_accuracy: 0.9674
Epoch 4/10
361/361 [==============================] - 20s 57ms/step - loss: 0.0823 - accuracy: 0.9742 - val_loss: 0.1194 - val_accuracy: 0.9626
Epoch 5/10
361/361 [==============================] - 20s 57ms/step - loss: 0.0686 - accuracy: 0.9782 - val_loss: 0.0632 - val_accuracy: 0.9800
Epoch 6/10
361/361 [==============================] - 21s 57ms/step - loss: 0.0605 - accuracy: 0.9806 - val_loss: 0.0813 - val_accuracy: 0.9736
Epoch 7/10
361/361 [==============================] - 21s 57ms/step - loss: 0.0521 - accuracy: 0.9834 - val_loss: 0.0558 - val_accuracy: 0.9835
Epo

In [8]:
for model_name, r in model_dict.items():
    print(f"{model_name} = {r['acc']}")

get_model_1 = 0.9002078771591187
get_model_2 = 0.8925849199295044
get_model_3 = 0.9313929080963135
get_stn_a_model_1 = 0.8800415992736816
get_stn_a_model_2 = 0.8670824766159058
get_stn_a_model_3 = 0.8801108598709106
get_stn_a_model_4 = 0.8910602927207947
get_stn_a_model_5 = 0.901386022567749
get_stn_a_model_6 = 0.9058905243873596
get_stn_a_model_7 = 0.8807345628738403
get_stn_a_model_8 = 0.9094941020011902
get_stn_a_model_9 = 0.9091476202011108
